This notebook is used to perform KIM to predict soil respirations from selected predictors using SRDB database.

In [1]:
# Libraries
from pathlib import Path
import pandas as pd

from kim.map import KIM
from kim.data import Data
from kim.mapping_model import MLP


# Read the data

In [2]:
# File and folder paths
f_data = Path('./selected_SRDB.csv')

In [3]:
df = pd.read_csv(f_data)
df.head()

,Site_ID,Latitude,Longitude,MAT,MAP,Annual_coverage,Soil_BD,Rs_annual
0,US-FIFG-600PPM,38.73,-120.80000,18.000,1000.00,0.25,1.14,647.0000
1,CA-CAR-CHMB3,49.85,-125.32000,8.600,1452.00,1.00,1.35,2200.0000
2,IE-OPR-PASTURE,52.85,-6.90000,9.400,824.00,1.00,1.42,1110.0000
3,CN-DHS-BF,23.16,112.51625,21.825,1746.75,1.00,0.85,1047.9125
4,US-NC-LFREQ,35.78,-75.90000,16.900,1270.00,1.00,0.08,1077.0000


In [4]:
# Predictors
x_keys = [
    "Latitude", "Longitude", "MAT", "MAP", "Annual_coverage", "Soil_BD"
]

# Predictands
y_keys = [
    "Rs_annual"
    # , "Ra_annual", "Rh_annual", "GPP"
]


In [5]:
x, y = df[x_keys].values, df[y_keys].values


In [6]:
x.shape, y.shape

((823, 6), (823, 1))

# Configurations

## Preliminary analysis configuration

In [7]:
seed_shuffle = 1234
f_data_save = Path("./results/data")


In [8]:
# Data configuration
data_params = {
    "xscaler_type": "minmax",
    "yscaler_type": "minmax",
}

# Sensitivity analysis configuration
sensitivity_params = {
    "method": "pc", "metric": "it-knn",
    "sst": True, "ntest": 100, "alpha": 0.05, "k": 3,
    "n_jobs": 50, "seed_shuffle": seed_shuffle,
    "verbose": 1
}


## Ensemble learning configuration

In [9]:
Ns_train = 600
Ns_val = 100
hidden_activation = 'sigmoid'
final_activation = 'leaky_relu'
seed_ens = 1024
seed_predict = 3636
seed_dl = 10
seed_model = 100
training_verbose = 1
n_models = 100
n_jobs = 50

f_kim_save1 = Path("./results/map_many2many")
f_kim_save2 = Path("./results/map_many2one")
f_kim_save3 = Path("./results/map_many2one_cond")


In [10]:
# Mapping parameters for each test below
map_configs = {
    "model_type": MLP,
    'n_model': n_models,
    'ensemble_type': 'ens_random',
    'model_hp_choices': {
        "depth": [1,3,5,6],
        "width_size": [3,6,10]
    },
    'model_hp_fixed': {
        "hidden_activation": hidden_activation,
        "final_activation": final_activation,
        "model_seed": seed_model
    },
    'optax_hp_choices': {
        'learning_rate': [0.01, 0.005, 0.003],
    },
    'optax_hp_fixed': {
        'nsteps': 300,
        'optimizer_type': 'adam',
    },
    'dl_hp_choices': {
    },
    'dl_hp_fixed': {
        'dl_seed': seed_dl,
        'num_train_sample': Ns_train,
        'num_val_sample': Ns_val,
        'batch_size': 64
    },
    'ens_seed': seed_ens,
    'training_parallel': True,
    'parallel_config': {
        'n_jobs': n_jobs, 
        'backend': 'loky',
        'verbose': 1
    },
    'device': None,
}

# Exploratory data analysis

In [11]:
data = Data(x, y, **data_params)
data.calculate_sensitivity(**sensitivity_params)
# Save the sensitivity analysis to disk
data.save(f_data_save)


Using the kNN-based information theoretic metrics ...
Performing pairwise analysis to remove insensitive inputs ...


100%|██████████| 6/6 [05:59<00:00, 59.91s/it] 


Performing conditional independence testing to remove redundant inputs ...
Thu Nov 28 19:16:28 2024:    ERROR: auth.munge: munge_decode() failed. Socket communication error


In [12]:
data.sensitivity_mask

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

In [13]:
data.cond_sensitivity_mask

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])

# Train the inverse mapping

In [14]:
# Initialize three diffferent KIMs
kim1 = KIM(data, map_configs, map_option='many2many')
kim2 = KIM(data, map_configs, mask_option="sensitivity", map_option='many2one')
kim3 = KIM(data, map_configs, mask_option="cond_sensitivity", map_option='many2one')

# Train the mappings
kim1.train()
kim2.train()
kim3.train()



 Performing ensemble training in parallel with 100 model configurations...



[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
100%|██████████| 300/300 [00:05<00:00, 50.35it/s]
[Parallel(n_jobs=50)]: Done 100 out of 100 | elapsed:   14.3s finished
[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
  0%|          | 0/300 [00:00<?, ?it/s]

Training completes.

 Performing ensemble training in parallel with 100 model configurations...



 82%|████████▏ | 246/300 [00:04<00:00, 66.67it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 98%|█████████▊| 295/300 [00:05<00:00, 65.75it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
 96%|█████████▋| 289/300 [00:04<00:00, 75.84it/s]

Training completes.

 Performing ensemble training in parallel with 100 model configurations...



100%|██████████| 300/300 [00:04<00:00, 60.19it/s]
[Parallel(n_jobs=50)]: Done 100 out of 100 | elapsed:  1.6min finished
[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
 77%|███████▋  | 232/300 [00:04<00:01, 62.82it/s]/global/cfs/cdirs/m1800/peishi/.conda/envs/kim/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 95%|█████████▌| 286/300 [00:05<00:00, 70.26it/s]

Training completes.


100%|██████████| 300/300 [00:05<00:00, 51.51it/s]
[Parallel(n_jobs=50)]: Done 100 out of 100 | elapsed:  1.5min finished


In [15]:
# Save 
kim1.save(f_kim_save1)
kim2.save(f_kim_save2)
kim3.save(f_kim_save3)
